In [192]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import re
import time
from datetime import date
from bs4 import BeautifulSoup, Comment
import math

# dpoy_table_list = []

# today = date.today()

# # input URL and use BeautifulSoup to parse through the page
# url = f'https://www.basketball-reference.com/awards/dpoy.html'
# soup = BeautifulSoup(requests.get(url).content, 'html.parser')

# # Grab the table element that has NBA player statistics
# dpoy_table = soup.select_one('#div_dpoy_NBA')

# # comment = allstar_table.find(text=lambda text: isinstance(text, Comment))
# # if comment.find("table") > 0:
# #         comment_soup = BeautifulSoup(comment, 'html.parser')
# #         table = comment_soup.find_all("table")

# # Grab data from table and put it into the list created above
# for tr in dpoy_table.select('tr:has(td)'):
#     tds = [td.get_text(strip=True) for td in tr.select('td')]
#     dpoy_table_list.append(tds)


# dpoy_table_list


In [198]:
team_codes_df = pd.read_csv('team_codes.csv')
games_attended_df = pd.read_csv('sports_attendance.csv')

home_teams_seen_list = games_attended_df['Home'].unique()
away_teams_seen_list = games_attended_df['Visitor'].unique()
all_teams_seen_list = list(home_teams_seen_list)

# for away_team in away_teams_seen_list:
#     all_teams_seen_list.append(away_team)

[all_teams_seen_list.append(x) for x in away_teams_seen_list if x not in all_teams_seen_list]

    
mlb_teams_list = [x for x in team_codes_df['MLB_Teams'].to_list() if str(x) != 'nan']
nba_teams_list = [x for x in team_codes_df['NBA_Teams'].to_list() if str(x) != 'nan']
nfl_teams_list = [x for x in team_codes_df['NFL_Teams'].to_list() if str(x) != 'nan']
nhl_teams_list = [x for x in team_codes_df['NHL_Teams'].to_list() if str(x) != 'nan']
mls_teams_list = [x for x in team_codes_df['MLS_Codes'].to_list() if str(x) != 'nan']

home_mlb_teams_seen_list = []
home_nba_teams_seen_list = []
home_nfl_teams_seen_list = []
home_nhl_teams_seen_list = []
home_mls_teams_seen_list = []

for team in home_teams_seen_list:
    if team in mlb_teams_list:
        home_mlb_teams_seen_list.append(team)
    elif team in nba_teams_list:
        home_nba_teams_seen_list.append(team)
    elif team in nfl_teams_list:
        home_nfl_teams_seen_list.append(team)
    elif team in nhl_teams_list:
        home_nhl_teams_seen_list.append(team)
    else: 
        team = team.replace(' ', '-')
        for mls_team in mls_teams_list:
            if team in mls_team:
                home_mls_teams_seen_list.append(team)
            else:
                continue

                
all_mlb_teams_seen_list = []
all_nba_teams_seen_list = []
all_nfl_teams_seen_list = []
all_nhl_teams_seen_list = []
all_mls_teams_seen_list = []

for team in all_teams_seen_list:
    if team in mlb_teams_list:
        all_mlb_teams_seen_list.append(team)
    elif team in nba_teams_list:
        all_nba_teams_seen_list.append(team)
    elif team in nfl_teams_list:
        all_nfl_teams_seen_list.append(team)
    elif team in nhl_teams_list:
        all_nhl_teams_seen_list.append(team)
    else: 
        team = team.replace(' ', '-')
        for mls_team in mls_teams_list:
            if team in mls_team:
                all_mls_teams_seen_list.append(team)
            else:
                continue
                
mlb_stadiums_visited = (len(home_mlb_teams_seen_list)/((len(mlb_teams_list))-1))
nba_stadiums_visited = (len(home_nba_teams_seen_list)/len(nba_teams_list))
nfl_stadiums_visited = (len(home_nfl_teams_seen_list)/len(nfl_teams_list))
nhl_stadiums_visited = (len(home_nhl_teams_seen_list)/len(nhl_teams_list))
mls_stadiums_visited = (len(home_mls_teams_seen_list)/len(mls_teams_list))

mlb_teams_seen = (len(all_mlb_teams_seen_list)/len(mlb_teams_list))
nba_teams_seen = (len(all_nba_teams_seen_list)/len(nba_teams_list))
nfl_teams_seen = (len(all_nfl_teams_seen_list)/len(nfl_teams_list))
nhl_teams_seen = (len(all_nhl_teams_seen_list)/len(nhl_teams_list))
mls_teams_seen = (len(all_mls_teams_seen_list)/len(mls_teams_list))

games_dict = {}

games_dict['League'] = ['MLB','NBA','NFL','NHL','MLS']
games_dict['League_Teams_Total'] = [((len(mlb_teams_list))-1),len(nba_teams_list),len(nfl_teams_list),len(nhl_teams_list),len(mls_teams_list)]
games_dict['Stadiums_Visited'] = [len(home_mlb_teams_seen_list),len(home_nba_teams_seen_list),len(home_nfl_teams_seen_list),len(home_nhl_teams_seen_list),len(home_mls_teams_seen_list)]
games_dict['%_Visited'] = [mlb_stadiums_visited, nba_stadiums_visited, nfl_stadiums_visited, nhl_stadiums_visited, mls_stadiums_visited]
games_dict['Teams_Seen'] = [len(all_mlb_teams_seen_list),len(all_nba_teams_seen_list),len(all_nfl_teams_seen_list),len(all_nhl_teams_seen_list),len(all_mls_teams_seen_list)]
games_dict['%_Teams_Seen'] = [mlb_teams_seen,nba_teams_seen,nfl_teams_seen,nhl_teams_seen,mls_teams_seen]


games_breakdown_df = pd.DataFrame(games_dict)
games_breakdown_df.loc[:, "%_Visited"] = games_breakdown_df["%_Visited"].map('{:.1%}'.format)
games_breakdown_df.loc[:, "%_Teams_Seen"] = games_breakdown_df["%_Teams_Seen"].map('{:.1%}'.format)
games_breakdown_df.set_index('League')


,League_Teams_Total,Stadiums_Visited,%_Visited,Teams_Seen,%_Teams_Seen
League,,,,,
MLB,30,27,90.0%,31,100.0%
NBA,30,8,26.7%,21,70.0%
NFL,32,1,3.1%,2,6.2%
NHL,32,2,6.2%,5,15.6%
MLS,28,1,3.6%,2,7.1%


,League,League_Teams_Total,Stadiums_Visited,%_Visited,Teams_Seen,%_Teams_Seen
0,MLB,30,27,0.900000,31,1.000000
1,NBA,30,8,0.266667,21,0.700000
2,NFL,32,1,0.031250,2,0.062500
3,NHL,32,2,0.062500,5,0.156250
4,MLS,28,1,0.035714,2,0.071429


27
8
1
2
1
['Arizona Diamondbacks', 'Atlanta Braves', 'Baltimore Orioles', 'Boston Red Sox', 'Chicago Cubs', 'Chicago White Sox', 'Cincinnati Reds', 'Cleveland Guardians', 'Colorado Rockies', 'Detroit Tigers', 'Houston Astros', 'Kansas City Royals', 'Los Angeles Angels', 'Los Angeles Dodgers', 'Miami Marlins', 'Florida Marlins', 'Milwaukee Brewers', 'Minnesota Twins', 'New York Mets', 'New York Yankees', 'Oakland Athletics', 'Philadelphia Phillies', 'Pittsburgh Pirates', 'San Diego Padres', 'San Francisco Giants', 'Seattle Mariners', 'St. Louis Cardinals', 'Tampa Bay Rays', 'Texas Rangers', 'Toronto Blue Jays', 'Washington Nationals']


53

False

In [53]:
today = date.today()

# dd/mm/YY
if int(today.strftime("%m")) > 6:
    current_season = int(today.strftime("%Y"))
else:
    current_season = (int(today.strftime("%Y")) - 1)

all_league_player_list = []
all_league_team_list = []
league_list = []


for item in all_league_table_list:
    counter = 3
    list_navigator = 3
    while list_navigator < 8:
        all_league_player_list.append(item[list_navigator])
        all_league_team_list.append(item[1])
        list_navigator = list_navigator + 1
        league_list.append(item[0])


cleaned_all_league_player_list = []
cleaned_all_league_team_list = []
cleaned_league_list = []
all_league_year_list = []

for item in all_league_player_list:
    if item == '':
        continue
    else:
        cleaned_all_league_player_list.append(item)

for item in all_league_team_list:
    if item == '':
        continue
    else:
        cleaned_all_league_team_list.append(item)

for item in league_list:
    if item == '':
        continue
    else:
        cleaned_league_list.append(item)

        
current_season = current_season + 1
        
for i in range(len(cleaned_all_league_team_list)):      
    if ((cleaned_all_league_team_list[i] == '1st') and (cleaned_all_league_team_list[i - 1] != '1st') and (cleaned_league_list[i] != 'ABA')):
        current_season = current_season - 1
        all_league_year_list.append(current_season)
    else:
        all_league_year_list.append(current_season)

        
all_league_df = pd.DataFrame(
    {'Year': all_league_year_list,
     'Player': cleaned_all_league_player_list,
     'Team': cleaned_all_league_team_list
    })

all_league_df


,Year,Player,Team
0,2022,Nikola JokićC,1st
1,2022,Giannis AntetokounmpoF,1st
2,2022,Jayson TatumF,1st
3,2022,Luka DončićG,1st
4,2022,Devin BookerG,1st
...,...,...,...
1015,1947,Frankie Baumholtz,2nd
1016,1947,Ernie Calverley,2nd
1017,1947,Chick Halbert,2nd
1018,1947,John Logan,2nd


,Year,Player,Team
620,1977,Kareem Abdul-JabbarC,1st
621,1977,Elvin HayesF,1st
622,1977,David ThompsonF,1st
623,1977,Pete MaravichG,1st
624,1977,Paul WestphalG,1st
625,1977,Bill WaltonC,2nd
626,1977,Julius ErvingF,2nd
627,1977,George McGinnisF,2nd
628,1977,George GervinG,2nd
629,1977,Jo Jo WhiteG,2nd


In [ ]:
final_hof_table_header_list = hof_table_header_list[1]
final_hof_table_header_list.pop(0)

hof_df = pd.DataFrame(hof_table_list, columns=final_hof_table_header_list)
#hof_df = hof_df.rename(columns={'0': 'Name', '1': 'Role'})
hof_df

In [ ]:
hof_names = hof_df['Name'].to_list()

# test1 = re.findall(r"^(.*?)WNBA", hof_names[0])
# test2 = re.findall(r"^(.*?)WNBA", hof_names[1])

cleaning_list = ['^(.*?)WNBA', '^(.*?)CBBplayer', '^(.*?)Coach', '^(.*?)CBBcoach', '^(.*?)Exec', '^(.*?)Ref', '^(.*?)Player', '^(.*?)Oly']
temp_hof_name_list = []

for name in hof_names:
#     temp_name_holder = 'a'
    cleaned_name = re.findall(r'^(.*?)\/', name)
    if cleaned_name == []:
        temp_hof_name_list.append(name)
    else:
        temp_hof_name_list.append(cleaned_name)

new_temp_name_list = []

for temp_name in temp_hof_name_list:
    if type(temp_name) == list:
        new_temp_name = str(temp_name[0])
        new_temp_name_list.append(new_temp_name)
    else:
        new_temp_name_list.append(temp_name)

cleaned_hof_name_list = []   

for temp_name in new_temp_name_list:
    name_holder = temp_name
    counter = 0 
    
    for string in cleaning_list:
        new_cleaned_name = re.findall(string, temp_name)

        if new_cleaned_name != []:
            cleaned_hof_name_list.append(new_cleaned_name)
            break
        elif (new_cleaned_name == [] and counter == len(cleaning_list)):
            cleaned_hof_name_list.append(name_holder)
        else:
            counter = counter + 1
            continue

            
final_temp_name_list = []

for final_name in cleaned_hof_name_list:
    if type(final_name) == list:
        final_name_string = str(final_name[0])
        final_temp_name_list.append(final_name_string)
    else:
        final_temp_name_list.append(final_name)

hof_df = pd.DataFrame(final_temp_name_list, columns = ["Player"])
hof_df


In [ ]:
new_temp_name_list = []

for temp_name in temp_hof_name_list:
    if type(temp_name) == list:
        new_temp_name = str(temp_name[0])
        new_temp_name_list.append(new_temp_name)
    else:
        new_temp_name_list.append(temp_name)

# test = str(temp_hof_name_list[1][0])
# print(type(test))
# test
new_temp_name_list

In [ ]:
dpoy_df = pd.DataFrame(dpoy_table_list)
dpoy_list = dpoy_df[1].to_list()

# dd/mm/YY
dpoy_year = int(today.strftime("%Y"))
if int(today.strftime("%m")) < 7:
    dpoy_year = dpoy_year - 1

dpoy_seen_df_list = []
for item in dpoy_list:
    dpoy_seen_df_list.append([item,dpoy_year])
    dpoy_year = dpoy_year - 1

dpoy_seen_df = pd.DataFrame(dpoy_seen_df_list, columns = ['Player', 'Year'])
agg_dpoy_seen_years_list = []
agg_dpoy_seen_list = []
player_dpoy_count_list = []

for player in dpoy_seen_df['Player'].unique():
    agg_dpoy_seen_list.append(player)
    agg_dpoy_seen_years_list.append(dpoy_seen_df.loc[dpoy_seen_df['Player'] == player]['Year'].to_list())
    player_dpoy_count_list.append(dpoy_seen_df.loc[dpoy_seen_df['Player'] == player]['Year'].count())
    
agg_dpoy_seen_df = pd.DataFrame(columns = ['Player','X_DPOY','Years'])
agg_dpoy_seen_df['Player'] = agg_dpoy_seen_list
agg_dpoy_seen_df['Years'] = agg_dpoy_seen_years_list
agg_dpoy_seen_df['X_DPOY'] = player_dpoy_count_list

agg_dpoy_seen_df